In [1]:
# Imports required libaries
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [2]:
# HTTP - request
# store a website in a variable
website = 'https://indiankanoon.org/browse/supremecourt/'

In [3]:
# make a request to a website
response = requests.get(website)

In [4]:
# check the status code
response.status_code

200

In [26]:
# Soup Object - get access to all the HTML elements
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
# soup

In [28]:
# Get the years from when the court records can be collected
years = soup.find('div', {'class':'info_indian_kanoon'}).get_text().split('      ')
years = [year.strip()[:4] for year in years]
years = years[:-1]
years

['1947',
 '1948',
 '1949',
 '1950',
 '1951',
 '1952',
 '1953',
 '1954',
 '1955',
 '1956',
 '1957',
 '1958',
 '1959',
 '1960',
 '1961',
 '1962',
 '1963',
 '1964',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1971',
 '1972',
 '1973',
 '1974',
 '1975',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021']

In [ ]:
# Create the links to further fetch the data 
year_base = 'https://indiankanoon.org/browse/supremecourt/'
year_links = [year_base + year + '/' for year in years]
year_links

In [30]:
# Inside every year there are the links for each month records
href="/search/?formInput=doctypes:supremecourt fromdate:1-7-1947 todate: 31-7-1947"
# /search/?formInput=doctypes:supremecourt%20fromdate:1-7-1947%20todate:%2031-7-1947

In [31]:
href.replace(' ', '%20')

'/search/?formInput=doctypes:supremecourt%20fromdate:1-7-1947%20todate:%2031-7-1947'

In [32]:
# Get the link for every month in the year
# Loop through every year and get the links for each month of that year
months_every_year = []
for year in year_links:
    website = year
    response = requests.get(website)
    soup = BeautifulSoup(response.content, 'html.parser')
    months = soup.find_all('div', {'class':'browselist'})
    months = ['https://indiankanoon.org' + a.find('a')['href'].replace(' ', '%20') for a in months]
    for month in months:
        months_every_year.append(month)
len(months_every_year)

900

In [33]:
months_every_year

['https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-1947%20todate:%2031-1-1947',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-1947%20todate:%2028-2-1947',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-1947%20todate:%2031-3-1947',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-1947%20todate:%2030-4-1947',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-1947%20todate:%2031-5-1947',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-1947%20todate:%2030-6-1947',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-1947%20todate:%2031-7-1947',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-1947%20todate:%2031-8-1947',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-1947%20todate:%2030-9-1947',
 

In [ ]:
# create a dataframe
sc_india = pd.DataFrame(columns=['Case Title', 'Year', 'Author', 'Judges', 'Document Id'])

# loop through entire list of month link and fetch the data
for month_year_link in months_every_year:
    # get the first page
    website_hm = month_year_link+'&pagenum=' + str(0)
    response = requests.get(website_hm)
    soup = BeautifulSoup(response.content, 'html.parser')

    # check the number of pages in the website
    try:
        number = int(soup.find('div', {'class':'results_middle'}).find('b').get_text().split(' ')[-1])
        print(number)
        i = 0;
        if number > 10:
            i = int(np.floor(number/10));
            print(i)
    except:
        i = -1
        
    for page_num in range(i+1) :  
        # check whether the page number is 0 , in case we already have soup for that
        if page_num > 0 :
            website = website_hm + str(page_num)
            response = requests.get(website)
            soup = BeautifulSoup(response.content, 'html.parser')
        # get the Document Id
        try:
            doc_id_list = soup.find_all('div', {'class' : 'result_title'})
            doc_id_list = [doc_id.find('a')['href'].split('/')[2] for doc_id in doc_id_list]
        except:
            doc_id_list = 'n/a'

        print(doc_id_list)
        page_num = page_num + 1

        for doc_id in doc_id_list:
            # get the doc link to collect more details
            website_doc = 'https://indiankanoon.org/doc/' + doc_id + '/'
            response = requests.get(website_doc)
            soup = BeautifulSoup(response.content, 'html.parser')

            # get the case title
            try:
                title = soup.find('div', {'class' : 'doc_title'}).get_text().strip()
            except:
                title = 'n/a'

            # get the author name
            try:
                author = soup.find('div', {'class' : 'doc_author'}).get_text().strip().replace('Author:', '')
            except:
                author = 'n/a'

            #  get the judge name
            try:
                bench = soup.find('div', {'class' : 'doc_bench'}).get_text().strip().replace('Bench:', '')
            except:
                bench = 'n/a'

            # get the year of the case
            try:
                year = title[-4:]
            except:
                year = 'n/a'


            sc_india = sc_india.append({'Case Title' : title,
                      'Year' : year,
                      'Author' : author,
                      'Judges' : bench,
                      'Document Id' : doc_id}, ignore_index = True)


In [61]:
sc_india.head()

,Case Title,Year,Author,Judges,Document Id
0,Hardeep Singh vs State Of Punjab & Ors on 10 J...,1947,. B Chauhan,"P Sathasivam, B.S. Chauhan, Ranjana Prakash D...",52754564
1,Sanjay Kumar vs State Of Bihar & Anr on 28 Jan...,1947,…………......................J.,"B.S. Chauhan, J. Chelameswar, M.Y. Eqbal",130498285
2,State Of T.Nadu Tr.Insp.Of Police vs N Suresh ...,1947,C K Prasad,"Chandramauli Kr. Prasad, M.Y. Eqbal",23015624
3,Union Of India & Ors vs Vasavi Co-Op. Housing ...,1947,…..………………………J.,"K.S. Radhakrishnan, A.K. Sikri",175295508
4,"Syed Sadiq Etc vs Divisional Manager,United In...",1947,………………………………………………………………………J.,"Sudhansu Jyoti Mukhopadhaya, V. Gopala Gowda",140442067


In [62]:
sc_india.shape

(28951, 5)

In [63]:
sc_india.to_csv('Supreme_Court_cases_India.csv')

In [65]:
sc_india['Case Title'][28950]

'Surendra Kumar vs Nathulal & Anr on 2 May, 2001'

In [ ]:
# Because of the ConnectionError We were able to collect the data in the above loop till may 2001 remainig is done below

In [6]:
# create a dataframe
sc_india = pd.DataFrame(columns=['Case Title', 'Year', 'Author', 'Judges', 'Document Id'])

for page_num in range(5, 9) :  
        # check whether the page number is 0 , in case we already have soup for that
        if page_num > 0 :
            website = 'https://indiankanoon.org/search/?formInput=doctypes%3A%20supremecourt%20fromdate%3A%201-5-2001%20todate%3A%2031-5-2001&pagenum=' + str(page_num)
            response = requests.get(website)
            soup = BeautifulSoup(response.content, 'html.parser')
        # get the Document Id
        try:
            doc_id_list = soup.find_all('div', {'class' : 'result_title'})
            doc_id_list = [doc_id.find('a')['href'].split('/')[2] for doc_id in doc_id_list]
        except:
            doc_id_list = 'n/a'

        print(doc_id_list)
        page_num = page_num + 1

        for doc_id in doc_id_list:
            # get the doc link to collect more details
            website_doc = 'https://indiankanoon.org/doc/' + doc_id + '/'
            response = requests.get(website_doc)
            soup = BeautifulSoup(response.content, 'html.parser')

            # get the case title
            try:
                title = soup.find('div', {'class' : 'doc_title'}).get_text().strip()
            except:
                title = 'n/a'

            # get the author name
            try:
                author = soup.find('div', {'class' : 'doc_author'}).get_text().strip().replace('Author:', '')
            except:
                author = 'n/a'

            #  get the judge name
            try:
                bench = soup.find('div', {'class' : 'doc_bench'}).get_text().strip().replace('Bench:', '')
            except:
                bench = 'n/a'

            # get the year of the case
            try:
                year = title[-4:]
            except:
                year = 'n/a'


            sc_india = sc_india.append({'Case Title' : title,
                      'Year' : year,
                      'Author' : author,
                      'Judges' : bench,
                      'Document Id' : doc_id}, ignore_index = True)



['390832', '301513', '1546671', '572091', '758268', '269366', '1554385', '181975', '1568693', '90330065']
['1542022', '1874234', '1694817', '952964', '1429461', '1158710', '489428', '1356442', '1038041', '1211865']
['1671657', '328826', '1268397', '146809709', '116899353', '116208964', '78300934', '3230973', '131226888', '1482443']
['57190634', '781188', '842959', '1474762']


In [7]:
sc_india.shape

(34, 5)

In [1]:
sc_india.head()

NameError: name 'sc_india' is not defined

In [9]:
sc_india.to_csv('Supreme_Court_cases_India1.csv')

In [11]:
sc1 = pd.read_csv('Supreme_Court_cases_India.csv')
sc1.shape

(28951, 6)

In [12]:
sc2 = pd.read_csv('Supreme_Court_cases_India1.csv')
sc2.shape

(34, 6)

In [35]:
sc_india = pd.concat([sc1, sc2], axis=0)
sc_india.shape

(28985, 6)

In [36]:
sc_india.to_csv('Supreme_Court_cases_India2.csv')

In [14]:
# We will continue the similar loop to collect the data after the month of May 2001

In [34]:
# The remaining years for ewhich the data has to be fetch
months_every_year = [
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2001%20todate:%2030-6-2001',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2001%20todate:%2031-7-2001',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2001%20todate:%2031-8-2001',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2001%20todate:%2030-9-2001',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2001%20todate:%2031-10-2001',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2001%20todate:%2030-11-2001',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2001%20todate:%2031-12-2001',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2002%20todate:%2031-1-2002',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2002%20todate:%2028-2-2002',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2002%20todate:%2031-3-2002',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2002%20todate:%2030-4-2002',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2002%20todate:%2031-5-2002',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2002%20todate:%2030-6-2002',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2002%20todate:%2031-7-2002',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2002%20todate:%2031-8-2002',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2002%20todate:%2030-9-2002',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2002%20todate:%2031-10-2002',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2002%20todate:%2030-11-2002',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2002%20todate:%2031-12-2002',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2003%20todate:%2031-1-2003',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2003%20todate:%2028-2-2003',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2003%20todate:%2031-3-2003',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2003%20todate:%2030-4-2003',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2003%20todate:%2031-5-2003',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2003%20todate:%2030-6-2003',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2003%20todate:%2031-7-2003',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2003%20todate:%2031-8-2003',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2003%20todate:%2030-9-2003',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2003%20todate:%2031-10-2003',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2003%20todate:%2030-11-2003',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2003%20todate:%2031-12-2003',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2004%20todate:%2031-1-2004',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2004%20todate:%2029-2-2004',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2004%20todate:%2031-3-2004',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2004%20todate:%2030-4-2004',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2004%20todate:%2031-5-2004',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2004%20todate:%2030-6-2004',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2004%20todate:%2031-7-2004',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2004%20todate:%2031-8-2004',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2004%20todate:%2030-9-2004',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2004%20todate:%2031-10-2004',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2004%20todate:%2030-11-2004',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2004%20todate:%2031-12-2004',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2005%20todate:%2031-1-2005',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2005%20todate:%2028-2-2005',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2005%20todate:%2031-3-2005',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2005%20todate:%2030-4-2005',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2005%20todate:%2031-5-2005',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2005%20todate:%2030-6-2005',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2005%20todate:%2031-7-2005',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2005%20todate:%2031-8-2005',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2005%20todate:%2030-9-2005',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2005%20todate:%2031-10-2005',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2005%20todate:%2030-11-2005',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2005%20todate:%2031-12-2005',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2006%20todate:%2031-1-2006',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2006%20todate:%2028-2-2006',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2006%20todate:%2031-3-2006',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2006%20todate:%2030-4-2006',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2006%20todate:%2031-5-2006',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2006%20todate:%2030-6-2006',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2006%20todate:%2031-7-2006',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2006%20todate:%2031-8-2006',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2006%20todate:%2030-9-2006',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2006%20todate:%2031-10-2006',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2006%20todate:%2030-11-2006',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2006%20todate:%2031-12-2006',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2007%20todate:%2031-1-2007',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2007%20todate:%2028-2-2007',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2007%20todate:%2031-3-2007',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2007%20todate:%2030-4-2007',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2007%20todate:%2031-5-2007',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2007%20todate:%2030-6-2007',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2007%20todate:%2031-7-2007',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2007%20todate:%2031-8-2007',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2007%20todate:%2030-9-2007',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2007%20todate:%2031-10-2007',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2007%20todate:%2030-11-2007',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2007%20todate:%2031-12-2007',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2008%20todate:%2031-1-2008',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2008%20todate:%2029-2-2008',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2008%20todate:%2031-3-2008',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2008%20todate:%2030-4-2008',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2008%20todate:%2031-5-2008',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2008%20todate:%2030-6-2008',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2008%20todate:%2031-7-2008',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2008%20todate:%2031-8-2008',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2008%20todate:%2030-9-2008',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2008%20todate:%2031-10-2008',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2008%20todate:%2030-11-2008',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2008%20todate:%2031-12-2008',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2009%20todate:%2031-1-2009',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2009%20todate:%2028-2-2009',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2009%20todate:%2031-3-2009',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2009%20todate:%2030-4-2009',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2009%20todate:%2031-5-2009',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2009%20todate:%2030-6-2009',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2009%20todate:%2031-7-2009',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2009%20todate:%2031-8-2009',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2009%20todate:%2030-9-2009',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2009%20todate:%2031-10-2009',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2009%20todate:%2030-11-2009',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2009%20todate:%2031-12-2009',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2010%20todate:%2031-1-2010',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2010%20todate:%2028-2-2010',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2010%20todate:%2031-3-2010',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2010%20todate:%2030-4-2010',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2010%20todate:%2031-5-2010',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2010%20todate:%2030-6-2010',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2010%20todate:%2031-7-2010',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2010%20todate:%2031-8-2010',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2010%20todate:%2030-9-2010',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2010%20todate:%2031-10-2010',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2010%20todate:%2030-11-2010',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2010%20todate:%2031-12-2010',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2011%20todate:%2031-1-2011',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2011%20todate:%2028-2-2011',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2011%20todate:%2031-3-2011',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2011%20todate:%2030-4-2011',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2011%20todate:%2031-5-2011',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2011%20todate:%2030-6-2011',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2011%20todate:%2031-7-2011',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2011%20todate:%2031-8-2011',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2011%20todate:%2030-9-2011',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2011%20todate:%2031-10-2011',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2011%20todate:%2030-11-2011',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2011%20todate:%2031-12-2011',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2012%20todate:%2031-1-2012',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2012%20todate:%2029-2-2012',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2012%20todate:%2031-3-2012',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2012%20todate:%2030-4-2012',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2012%20todate:%2031-5-2012',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2012%20todate:%2030-6-2012',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2012%20todate:%2031-7-2012',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2012%20todate:%2031-8-2012',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2012%20todate:%2030-9-2012',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2012%20todate:%2031-10-2012',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2012%20todate:%2030-11-2012',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2012%20todate:%2031-12-2012',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2013%20todate:%2031-1-2013',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2013%20todate:%2028-2-2013',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2013%20todate:%2031-3-2013',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2013%20todate:%2030-4-2013',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2013%20todate:%2031-5-2013',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2013%20todate:%2030-6-2013',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2013%20todate:%2031-7-2013',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2013%20todate:%2031-8-2013',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2013%20todate:%2030-9-2013',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2013%20todate:%2031-10-2013',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2013%20todate:%2030-11-2013',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2013%20todate:%2031-12-2013',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2014%20todate:%2031-1-2014',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2014%20todate:%2028-2-2014',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2014%20todate:%2031-3-2014',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2014%20todate:%2030-4-2014',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2014%20todate:%2031-5-2014',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2014%20todate:%2030-6-2014',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2014%20todate:%2031-7-2014',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2014%20todate:%2031-8-2014',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2014%20todate:%2030-9-2014',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2014%20todate:%2031-10-2014',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2014%20todate:%2030-11-2014',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2014%20todate:%2031-12-2014',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2015%20todate:%2031-1-2015',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2015%20todate:%2028-2-2015',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2015%20todate:%2031-3-2015',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2015%20todate:%2030-4-2015',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2015%20todate:%2031-5-2015',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2015%20todate:%2030-6-2015',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2015%20todate:%2031-7-2015',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2015%20todate:%2031-8-2015',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2015%20todate:%2030-9-2015',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2015%20todate:%2031-10-2015',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2015%20todate:%2030-11-2015',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2015%20todate:%2031-12-2015',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2016%20todate:%2031-1-2016',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2016%20todate:%2029-2-2016',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2016%20todate:%2031-3-2016',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2016%20todate:%2030-4-2016',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2016%20todate:%2031-5-2016',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2016%20todate:%2030-6-2016',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2016%20todate:%2031-7-2016',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2016%20todate:%2031-8-2016',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2016%20todate:%2030-9-2016',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2016%20todate:%2031-10-2016',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2016%20todate:%2030-11-2016',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2016%20todate:%2031-12-2016',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2017%20todate:%2031-1-2017',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2017%20todate:%2028-2-2017',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2017%20todate:%2031-3-2017',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2017%20todate:%2030-4-2017',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2017%20todate:%2031-5-2017',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2017%20todate:%2030-6-2017',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2017%20todate:%2031-7-2017',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2017%20todate:%2031-8-2017',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2017%20todate:%2030-9-2017',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2017%20todate:%2031-10-2017',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2017%20todate:%2030-11-2017',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2017%20todate:%2031-12-2017',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2018%20todate:%2031-1-2018',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2018%20todate:%2028-2-2018',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2018%20todate:%2031-3-2018',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2018%20todate:%2030-4-2018',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2018%20todate:%2031-5-2018',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2018%20todate:%2030-6-2018',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2018%20todate:%2031-7-2018',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2018%20todate:%2031-8-2018',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2018%20todate:%2030-9-2018',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2018%20todate:%2031-10-2018',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2018%20todate:%2030-11-2018',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2018%20todate:%2031-12-2018',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2019%20todate:%2031-1-2019',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2019%20todate:%2028-2-2019',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2019%20todate:%2031-3-2019',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2019%20todate:%2030-4-2019',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2019%20todate:%2031-5-2019',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2019%20todate:%2030-6-2019',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2019%20todate:%2031-7-2019',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2019%20todate:%2031-8-2019',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2019%20todate:%2030-9-2019',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2019%20todate:%2031-10-2019',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2019%20todate:%2030-11-2019',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2019%20todate:%2031-12-2019',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2020%20todate:%2031-1-2020',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2020%20todate:%2029-2-2020',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2020%20todate:%2031-3-2020',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2020%20todate:%2030-4-2020',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2020%20todate:%2031-5-2020',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2020%20todate:%2030-6-2020',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2020%20todate:%2031-7-2020',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2020%20todate:%2031-8-2020',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2020%20todate:%2030-9-2020',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2020%20todate:%2031-10-2020',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2020%20todate:%2030-11-2020',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2020%20todate:%2031-12-2020',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-1-2021%20todate:%2031-1-2021',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-2-2021%20todate:%2028-2-2021',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-3-2021%20todate:%2031-3-2021',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-4-2021%20todate:%2030-4-2021',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-5-2021%20todate:%2031-5-2021',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-6-2021%20todate:%2030-6-2021',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-7-2021%20todate:%2031-7-2021',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-8-2021%20todate:%2031-8-2021',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-9-2021%20todate:%2030-9-2021',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-10-2021%20todate:%2031-10-2021',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-11-2021%20todate:%2030-11-2021',
 'https://indiankanoon.org/search/?formInput=doctypes:supremecourt%20fromdate:1-12-2021%20todate:%2031-12-2021'
]

In [ ]:
# create a dataframe --> Same code as above
sc_india = pd.DataFrame(columns=['Case Title', 'Year', 'Author', 'Judges', 'Document Id'])

# loop through entire list of month link and fetch the data
for month_year_link in months_every_year:
    # get the first page
    website_hm = month_year_link+'&pagenum=' + str(0)
    response = requests.get(website_hm)
    soup = BeautifulSoup(response.content, 'html.parser')

    # check the number of pages in the website
    try:
        number = int(soup.find('div', {'class':'results_middle'}).find('b').get_text().split(' ')[-1])
        print(number)
        i = 0;
        if number > 10:
            i = int(np.floor(number/10));
            print(i)
    except:
        i = -1
        
    for page_num in range(i+1) :  
        # check whether the page number is 0 , in case we already have soup for that
        if page_num > 0 :
            website = website_hm + str(page_num)
            response = requests.get(website)
            soup = BeautifulSoup(response.content, 'html.parser')
        # get the Document Id
        try:
            doc_id_list = soup.find_all('div', {'class' : 'result_title'})
            doc_id_list = [doc_id.find('a')['href'].split('/')[2] for doc_id in doc_id_list]
        except:
            doc_id_list = 'n/a'

        print(doc_id_list)
        page_num = page_num + 1

        for doc_id in doc_id_list:
            # get the doc link to collect more details
            website_doc = 'https://indiankanoon.org/doc/' + doc_id + '/'
            response = requests.get(website_doc)
            soup = BeautifulSoup(response.content, 'html.parser')

            # get the case title
            try:
                title = soup.find('div', {'class' : 'doc_title'}).get_text().strip()
            except:
                title = 'n/a'

            # get the author name
            try:
                author = soup.find('div', {'class' : 'doc_author'}).get_text().strip().replace('Author:', '')
            except:
                author = 'n/a'

            #  get the judge name
            try:
                bench = soup.find('div', {'class' : 'doc_bench'}).get_text().strip().replace('Bench:', '')
            except:
                bench = 'n/a'

            # get the year of the case
            try:
                year = title[-4:]
            except:
                year = 'n/a'


            sc_india = sc_india.append({'Case Title' : title,
                      'Year' : year,
                      'Author' : author,
                      'Judges' : bench,
                      'Document Id' : doc_id}, ignore_index = True)


87
8
['441929', '1974324', '1206100', '1701224', '1917283', '1389589', '1791620', '1600445', '1766988', '605834']
['1323993', '1483262', '1815324', '1706713', '641383', '870673', '989798', '592428', '454130', '1973429']
['1130828', '53461189', '1030147', '406616', '840688', '1248667', '1691248', '455820', '1865386', '893566']
['839959', '902306', '1442809', '1242357', '147864248', '253640', '1138910', '1945324', '1852811', '791266']
['735843', '199374', '1752197', '739978', '162668', '250025', '97268', '1132839', '1679705', '1117516']
['596603', '812915', '865927', '874542', '1439141', '1670628', '1210590', '135784847', '1725787', '656475']
['143274', '1352674', '1665059', '795107', '515955', '1928163', '678728', '139810459', '104338615', '23220799']
['293351', '528034', '764208', '1853494', '47468078', '125069', '661966', '740631', '1788027', '960937']
['628818', '15527419', '116745', '1305292', '1518671', '1989982', '1438022']
141
14
['508534', '277653', '1761200', '927219', '1496684

['749742', '1792943', '1996721', '301037', '1254801', '993795', '21170430', '206287', '970762', '1099140']
['145448', '196263', '1530660', '1673016', '1459701', '282754', '162840', '1242447', '710128', '39456322']
['1356846', '1641482', '286456', '475164', '475581', '1065188', '177671', '1957485', '1847242', '1956863']
['455271', '1437733', '1394316', '1812884', '693886', '1847924', '761927', '1814848', '644156', '185181']
['541944', '1418361', '585733', '1924046', '812650', '758361', '1607001', '1803247', '672567', '1507100']
['1884042', '1446874', '484360', '531290', '670237', '619634', '1822564', '1583287', '346249', '1952539']
['34996', '1386238', '1022228', '786018', '1466703', '1639514', '1508763', '566207', '601401', '764346']
['92961', '1123496', '1624977', '1254904', '1013286', '720190', '278736', '38416', '1146355', '1965090']
['75569', '1001827', '422999', '1655775', '3047', '375231', '1375972', '1470962', '1878336', '1024836']
['884832', '453869', '1524756', '1569932', '588

['12702', '536185', '1448089', '575561', '151948', '1470925', '198134513', '1929825', '92014', '408412']
['663701', '456558', '1554127', '816451', '719805', '1417133', '192478', '454358', '1562828', '872773']
['1737067', '1466881', '307707', '623978', '1077600', '269161', '62111', '576879', '185406', '868316']
['1116844', '1083055', '1770420', '994206', '192924807', '1054403', '152789', '191730', '527354', '1569287']
['992152', '1601492', '1517433', '1086382', '1864208', '1641639', '554486', '768509']
117
11
['512761', '1737714', '137587', '1250057', '332673', '469763', '268670', '907049', '1564738', '1199248']
['1389262', '1885523', '858067', '159992', '1136418', '1043321', '1489761', '1009046', '181156', '189253']
['265997', '282585', '736558', '1705114', '434509', '1982038', '1911675', '344330', '1063572', '1020649']
['570393', '976161', '408620', '344937', '462447', '1854196', '215097', '921223', '226350', '342770']
['1539576', '829043', '1180569', '609016', '30094296', '1007126', 

['247119', '560467', '819416', '171202', '60084', '1884869', '1527467', '1772440', '1438364', '430610']
['722447', '1692733', '711257', '129015723', '162668847', '1180332', '139179597', '14206', '625911', '1490050']
['729429', '1296109', '1257527', '1361947', '874766', '123749', '1561978', '476719', '84576626', '1673629']
['1074893', '1023128', '1174790', '1319959', '1955485', '909012', '1378399', '402095', '1667160', '1127041']
['1830467', '146109', '239619', '1051880', '1159249', '1825359', '653601', '1666629', '1061456', '448489']
['1875732', '609274', '438137', '633652', '25568670', '1041296', '1589072', '1904049', '867268', '813173']
['81562724', '1485260', '90619362', '1528019', '171288263', '1759886', '51466', '650346', '197420', '1502393']
['1774434', '139000', '1485916', '609377', '1025029', '119878', '567993', '1816903', '1134563', '806556']
['121264', '1179529', '748368', '210345', '1918252', '1667781', '262633', '1807019', '641426', '632424']
['304914', '250859', '151306694

['464502', '179157436', '1457362', '1955995', '608532', '1977191', '1243114', '1142960', '1474423', '443550']
['171714', '77281346', '67558957', '855844', '1962501', '441621', '1640871', '1457677', '56112100', '352877']
['1061863', '1840164', '51611', '403780']
129
12
['1147125', '1107385', '1941716', '1797151', '1603336', '39828', '309064', '52511', '1774813', '1199911']
['1305345', '846026', '105580756', '447378', '1859141', '1853683', '926636', '1645305', '745068', '88823']
['337165', '249842', '686483', '837614', '1929751', '1821668', '1642159', '1309066', '46627', '6908672']
['343975', '1389603', '1433629', '1356651', '1382148', '1487347', '936043', '1793348', '1005932', '213147']
['725952', '1201336', '1640521', '1164866', '1200240', '1667490', '1597415', '676701', '1983487', '622779']
['904716', '1312126', '1354141', '1005422', '1257919', '9699283', '51359270', '157931285', '263706', '1925724']
['9701717', '1075561', '1453639', '434494', '1658376', '463738', '444227', '1396445',

...

['1218465', '854142', '1441058', '32806', '1731904', '1957148', '3544', '662814', '900898', '820492']
['1163102', '118489', '1767920', '1193535', '758534', '1107084', '92569', '708762', '1132520', '1216908']
['464821', '81358', '1755546', '300122', '154650', '1190958', '1954341', '319225', '1823231', '1865300']
['1477014', '804937', '196315128', '1135333', '909293', '462095', '601422', '1655833', '1829303', '499395']
['431546', '416821', '172527', '1805812', '385396', '916743', '1296228', '33712', '181466178', '1692662']
['1864574', '1049365', '737329', '65151197', '377410', '115360', '1794785', '1004370', '1188931', '100474']
['906773', '111117', '925586', '21201', '1204789', '573113', '992842', '63458282', '832862', '198893']
['202694', '203644', '683949', '1536287', '199741', '1807998', '867151', '359136', '772633', '1126743']
['1918896', '969490', '1965074', '191751903', '714265', '1126058', '1575112', '1329074', '258014', '581752']
['1100983', '643273', '1850285', '1714168', '1243

['139431', '431047', '276687', '218589', '133590', '485414', '65341', '1973253', '373387', '33313']
['1672796', '486451', '390860', '862418', '1571159', '1342712', '432350', '102563', '97751', '1187651']
['1982545', '1309882', '599398', '76719', '206888', '267586', '552186', '1934910', '932444', '422422']
['51185', '293836', '172609', '1430149', '525251', '482429', '1663677', '159748', '1215927', '1787541']
['1163969', '86190', '46415601', '585289', '397409', '995274', '1355629', '431237', '171955395', '853135']
['1255779', '49713', '163257896', '26085867', '643386', '169459684', '149051', '1660794', '1190254', '117098']
['1570855', '83860580', '556154', '1772002', '305319', '539661', '1321464', '665139', '469436', '641595']
['237563', '1258467', '1525379', '1346507', '251071', '314465', '1932179', '1546724', '1028806', '1217308']
['244807', '1105587', '1413944', '1964178', '1155939', '1042934', '1552598', '998863', '378956', '247263']
['18417', '1410302', '1518299', '604694', '714575'

['397156', '1028679', '379518', '327653', '1899551', '415046', '28474425']
259
25
['1703207', '1006308', '1610804', '1357349', '822220', '65700', '732230', '1159199', '173042', '63489929']
['1936939', '994091', '1792834', '1864824', '1361828', '226115', '1917158', '734499', '1971273', '1237261']
['1019462', '456885', '1705954', '21622', '408208', '284034', '559313', '1239532', '1785819', '455201']
['822820', '349982', '251771', '1570070', '1211064', '621587', '1759478', '899629', '295313', '108954']
['259597', '1823635', '1878735', '1775876', '1272244', '125684', '533374', '931239', '1863541', '607047']
['1253479', '88498', '1655298', '1527801', '670222', '1818558', '1428514', '841641', '1129587', '28565']
['287415', '1719437', '1156542', '626055', '1714594', '469092', '1235432', '115656', '495524', '410662']
['24680', '1440927', '451899', '12899', '507467', '764066', '1139132', '1010860', '131396', '943100']
['1839673', '463461', '1297691', '838576', '117875', '98872883', '1603142', '

['172615232', '146849', '93617247', '161649337', '134958329', '1361274', '952992', '81602448', '58699732', '1843187']
['308272', '192434609', '263362', '1989681', '154965922', '1081226', '639896', '138319067', '23382814', '1560465']
['13668', '1143942', '1496144', '1202384', '780988', '452228', '1405534', '864779', '747515', '1857782']
['71872878', '959574', '1128582', '970401', '1148309', '125493', '137950959', '510501', '482800', '814303']
['493770', '104691022', '165244', '981939', '77495', '1050822', '867338', '70591', '391402', '105158']
['98569', '1746630', '124105008', '778187']
212
21
['393489', '1962587', '26187086', '1617540', '1268797', '1308555', '1734467', '1693089', '687286', '843923']
['581087', '1351451', '859906', '883883', '1867697', '481558', '1380511', '885483', '983426', '241174']
['1663039', '855552', '1118593', '74588', '366310', '706731', '1951809', '287972', '35890941', '1917539']
['539326', '1763324', '1679578', '845531', '656084', '1407332', '324967', '284849

['1190269', '699421', '1012533', '1393412', '280492', '1159495', '1557145', '391042', '1930868', '1508800']
['964080', '237862', '127084', '635116', '1376601', '440375', '665950', '1778959', '756216', '111805889']
['198675', '1252341', '199619', '437113', '1464305', '1213344', '1670966', '716921', '489963', '1513714']
['1909715', '833083', '339716', '119714', '1949724', '170106202', '665493', '668426', '729513', '368303']
['1128549', '1829285', '1049272', '59140', '771902', '1615637', '75961099', '627077', '736387', '588511']
['1801623', '1827220', '351022', '1252814', '843467', '29838', '300835', '755889', '1408862', '1362769']
['1827340', '67372942', '1071311', '966820', '1423769', '903069', '1735247', '209832', '1455497', '1585626']
['44956', '1114512', '36237', '788103', '1005651', '1538132', '533636', '185557844', '1112963', '1855043']
['1901267', '1044746', '28912184', '1159032', '111537', '21175', '1193748', '153743493', '278493', '1893643']
['198899679', '1465435', '1770580', '

In [ ]:
sc_india.head()

In [ ]:
sc_india.shape()

In [ ]:
sc_india.to_csv('Supreme_Court_cases_India3.csv')

In [1]:
# Read the csv created

In [7]:
# Remove the unknown columns - Unnamed: 0
sc1 = pd.read_csv('Supreme_Court_cases_India2.csv')
sc2 = pd.read_csv('Supreme_Court_cases_India3.csv')
sc1.drop(sc1.filter(regex="Unname"),axis=1, inplace=True)
sc2.drop(sc2.filter(regex="Unname"),axis=1, inplace=True)

In [8]:
sc1.head()

,Case Title,Year,Author,Judges,Document Id
0,Hardeep Singh vs State Of Punjab & Ors on 10 J...,1947,. B Chauhan,"P Sathasivam, B.S. Chauhan, Ranjana Prakash D...",52754564
1,Sanjay Kumar vs State Of Bihar & Anr on 28 Jan...,1947,…………......................J.,"B.S. Chauhan, J. Chelameswar, M.Y. Eqbal",130498285
2,State Of T.Nadu Tr.Insp.Of Police vs N Suresh ...,1947,C K Prasad,"Chandramauli Kr. Prasad, M.Y. Eqbal",23015624
3,Union Of India & Ors vs Vasavi Co-Op. Housing ...,1947,…..………………………J.,"K.S. Radhakrishnan, A.K. Sikri",175295508
4,"Syed Sadiq Etc vs Divisional Manager,United In...",1947,………………………………………………………………………J.,"Sudhansu Jyoti Mukhopadhaya, V. Gopala Gowda",140442067


In [9]:
sc2.head()

,Case Title,Year,Author,Judges,Document Id
0,Hiten P. Dalal vs Bratindranath Banerjee on 11...,2001,Ruma Pal,"Ruma Pall, Brijesh Kumar, B.N.Kirpal",441929
1,T.T.Antony vs State Of Kerala & Ors on 12 July...,2001,S S Quadri,"S.S.M.Quadri, S.N.Phukan",1974324
2,Vedabai @ Vaijayanatabai Baburao ... vs Shanta...,2001,NaN,"Syed Shah Quadri, S.N. Phukan",1206100
3,Shyam Sunder And Others vs Ram Kumar And Anoth...,2001,V N Khare,"N.S.Hegde, Y.K.Sabharwal, S.V.Patil",1701224
4,Commissioner Of Income-Tax vs Suresh Chandra M...,2001,NaN,"S Bharucha, Y Sabharwal, B Kumar",1917283


In [11]:
sc1.shape

(28985, 5)

In [12]:
sc2.shape

(24438, 5)

In [13]:
sc1.columns == sc2.columns

array([ True,  True,  True,  True,  True])

In [14]:
# Merge  and save in one data frame
sc_india = pd.concat([sc1, sc2], axis=0,ignore_index = True)
sc_india.shape

(53423, 5)

In [15]:
sc_india.tail()

,Case Title,Year,Author,Judges,Document Id
53418,Anand Kumar Tiwari vs High Court Of Madhya Pra...,2021,L. Nageswara Rao,"L. Nageswara Rao, Aniruddha Bose",105056923
53419,Lachhmi Narain Singh (D) Thr. ... vs Sarjug Si...,2021,Hrishikesh Roy,"Sanjay Kishan Kaul, Hrishikesh Roy",185528426
53420,Hemraj Ratnakar Salian vs Hdfc Bank Ltd. on 17...,2021,S. Abdul Nazeer,"S. Abdul Nazeer, Krishna Murari",172174360
53421,The State Of Uttar Pradesh vs Uttam Singh on 3...,2021,Sanjay Kishan Kaul,"Sanjay Kishan Kaul, Hrishikesh Roy",65485129
53422,Rajinder Goel vs High Court Of Punjab And Hary...,2021,Uday Umesh Lalit,"Uday Umesh Lalit, Ajay Rastogi",115649227


In [ ]:
sc_india.to_csv('Supreme_Court_cases_India_all.csv')

In [16]:
# Load the data frame
sc_india = pd.read_csv('Supreme_Court_cases_India_all.csv')
sc_india.drop(sc_india.filter(regex="Unname"),axis=1, inplace=True)
sc_india.head()

,Case Title,Year,Author,Judges,Document Id
0,Hardeep Singh vs State Of Punjab & Ors on 10 J...,1947,. B Chauhan,"P Sathasivam, B.S. Chauhan, Ranjana Prakash D...",52754564
1,Sanjay Kumar vs State Of Bihar & Anr on 28 Jan...,1947,…………......................J.,"B.S. Chauhan, J. Chelameswar, M.Y. Eqbal",130498285
2,State Of T.Nadu Tr.Insp.Of Police vs N Suresh ...,1947,C K Prasad,"Chandramauli Kr. Prasad, M.Y. Eqbal",23015624
3,Union Of India & Ors vs Vasavi Co-Op. Housing ...,1947,…..………………………J.,"K.S. Radhakrishnan, A.K. Sikri",175295508
4,"Syed Sadiq Etc vs Divisional Manager,United In...",1947,………………………………………………………………………J.,"Sudhansu Jyoti Mukhopadhaya, V. Gopala Gowda",140442067


In [17]:
sc_india.head()

,Case Title,Year,Author,Judges,Document Id
0,Hardeep Singh vs State Of Punjab & Ors on 10 J...,1947,. B Chauhan,"P Sathasivam, B.S. Chauhan, Ranjana Prakash D...",52754564
1,Sanjay Kumar vs State Of Bihar & Anr on 28 Jan...,1947,…………......................J.,"B.S. Chauhan, J. Chelameswar, M.Y. Eqbal",130498285
2,State Of T.Nadu Tr.Insp.Of Police vs N Suresh ...,1947,C K Prasad,"Chandramauli Kr. Prasad, M.Y. Eqbal",23015624
3,Union Of India & Ors vs Vasavi Co-Op. Housing ...,1947,…..………………………J.,"K.S. Radhakrishnan, A.K. Sikri",175295508
4,"Syed Sadiq Etc vs Divisional Manager,United In...",1947,………………………………………………………………………J.,"Sudhansu Jyoti Mukhopadhaya, V. Gopala Gowda",140442067


In [18]:
sc_india.shape

(53423, 5)

In [19]:
# Conclusion - DataFrame is created that contains the basic details of the Document along with document Id